## Scraping Fed information from MarketWatch

In [26]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from collections import defaultdict
import re

chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

#### Selenium cheat sheet

In [27]:
#x = driver.find_element_by_name('name')
#x.send_keys('type a string' or Keys.RETURN)

#button = driver.find_element_by_xpath('//tag[text()="TEXT"]')
#button.click()

#button2 = driver.find_element_by_xpath('//tag[contains(@class_style_or_other_tag, "=STRING")]')
#button2.click()
#sometimes has /b at the end

In [39]:
url = 'http://www.marketwatch.com'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [40]:
#Search 'yellen'
path = '//a[@class="btn btn--outline btn--search"]'
driver.find_element_by_xpath(path).click()
search = driver.find_element_by_class_name('search__input')
search.send_keys('yellen')
search.send_keys(Keys.RETURN)

#Advanced Searcb
driver.find_element_by_id('refinesearchtoggle').click()
driver.find_element_by_xpath('//input[@type="checkbox"]').click()

#Set Subject
search_mode = '//select[@id="m"]/option[@value="Subject"]'
driver.find_element_by_xpath(search_mode).click()

#Results Per Page
results = '//select[@id="rpp"]/option[@value="50"]'
driver.find_element_by_xpath(results).click()

#Economic News
search_mode = '//select[@id="mp"]/option[@value="ECAT"]'
driver.find_element_by_xpath(search_mode).click()

#Set date
date = driver.find_element_by_id('bdv')
date.click()
date.send_keys('09/28/2017')

#Search
driver.find_element_by_xpath('//input[@value="Search"]').click()
current_url = driver.current_url

In [46]:
soup = BeautifulSoup(requests.get(current_url).text, 'html.parser')

In [47]:
links = dict()
pages = soup.find_all(class_='searchresult')
for div in pages:
    attrs = dict()
    attrs['url'] = div.a['href']
    attrs['date'] = div.next_sibling.span.text
    links[div.a.text] = attrs

In [48]:
links

{'2-year Treasury yield is highest since 2008': {'date': '4:49 p.m. Sept. 21, 2017',
  'url': 'http://www.marketwatch.com/story/2-year-treasury-yield-holds-around-highest-level-since-2008-2017-09-21'},
 '2-year Treasury yield records longest string of gains in 2 months': {'date': '4:31 p.m. Sept. 14, 2017',
  'url': 'http://www.marketwatch.com/story/treasury-yields-tick-higher-as-consumer-inflation-data-looms-2017-09-14'},
 'A sexist view of marriage and why the world is running out of antibiotics': {'date': '5:25 p.m. Sept. 20, 2017',
  'url': 'http://www.marketwatch.com/story/a-sexist-view-of-marriage-and-why-the-world-is-running-out-of-antibiotics-2017-09-20'},
 'Currency war talk restarts after Jackson Hole': {'date': '4:06 p.m. Aug. 30, 2017',
  'url': 'http://www.marketwatch.com/story/currency-war-talk-restarts-after-jackson-hole-2017-08-30'},
 'Dollar holds on to gains after Yellen reiterates case for rate hikes': {'date': '4:12 p.m. Sept. 26, 2017',
  'url': 'http://www.marketw

In [7]:
def search_links(links):
    for name, attrs in links.items():
        driver = webdriver.Chrome(chromedriver)
        driver.get(attrs['url'])
        soup = BeautifulSoup(requests.get(attrs['url']).text, 'html.parser')
        paragraphs = soup.find(id='article-body').find_all('p')
        attrs.update({'content': [' '.join(t.split()) for t in [''.join(p.find_all(text=True)) for p in paragraphs if bool(p.findChildren('strong')) == False]]})
        driver.close
    return links
articles = search_links(links)

In [8]:
articles

{'10-year Treasury yield jumps most in six months as details of tax reform emerge': {'content': ['U.S. Treasury prices retreated further on Wednesday, driving yields higher, after the White House unveiled further details of its plan to overhaul the tax code, reviving the so-called reflation trade that was underpinned by hopes that President Donald Trump’s fiscal agenda would stoke inflation pressures.',
   'Investors also sold their holdings of government paper after Federal Reserve Chairwoman Janet Yellen on Tuesday outlined the Fed’s reasoning for an additional rate increase this year. Fed funds futures traders are pricing close to an 80% chance of another rate increase in 2017, compared with about 38% a month ago, according to CME Group data.',
   'Long-dated bonds led the action, with the 10-year note yield and the 30-year bond yield posting their biggest one day jump since March. 1. The 10-year Treasury yield TMUBMUSD10Y, +0.00% climbed 8 basis points to 2.309%. Likewise, the 30-y

In [9]:
for name, attrs in articles.items():
    attrs['content'] = ''.join(attrs['content'])

In [14]:
for name, attrs in articles.items():
    attrs['content'] = re.sub(r'\.([A-Z])', r'. \1', attrs['content'])

In [17]:
for a in articles.keys():
    print(a)

Gold edges up from a 6-week low
European stocks notch 2-month high, but bank stocks pare heftier gain
U.K. stocks log slight gains as bank shares rally on interest-rate hopes
Is Trump’s tax overhaul driving the selloff in Treasurys?
Fed’s Rosengren backs ‘regular and gradual’ interest-rate hikes
Dow snaps 4-day losing streak; financials, tax-cut hopes provide lift
Dollar index hits 1-month high as Republican tax-plan details emerge
10-year Treasury yield jumps most in six months as details of tax reform emerge
Russell 2000 jumps to record, on track for best day since June
Gold marks fresh September low as dollar gains
Fed’s Bullard sees no need to raise interest rates anytime soon
European stocks jump to 10-week high as banks rally
FTSE 100 breaks 2-day losing run as pound slides
Financial ETF surges to a decade high as Fed, data help boost Treasury yields
The ‘old school’ reason why you haven’t missed the tech melt-up


In [11]:
#econ_politics_selector = '//a[contains(text(), "Economy & Politics")]'
#driver.find_element_by_xpath(econ_politics_selector).click()

In [12]:
#the_fed = '//a[@href="/federal-reserve"]'
#driver.find_element_by_xpath(the_fed).click()